<h1><center> IT550: Information Retrieval<h1>
<h2> <center> Assignment 6: Query Expansion and Reformulation </h2>


<h4> <center> <b>Name : Arjun Vankani </h4>
<h4> <b> <center>ID : 202211036 </h4>

**Instilling python terrier**

In [1]:
!pip install python-terrier

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.4/104.4 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 51.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.6/69.6 kB 8.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.0/318.0 kB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.8/48.8 kB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.4/287.4 kB 27.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.1/45.1 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 55.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ..

**Importing necessary library**

In [2]:
import pandas as pd
import numpy as np
import pyterrier as pt

**Intilize pyterrrier**

In [4]:
if not pt.started():
  pt.init()

**Load dataset**

**The Vaswani NPL corpus is a small test collection of 11,000 abstracts has been used by the Glasgow IR group for many years**

In [5]:
data = pt.get_dataset("vaswani")

In [6]:
data

RemoteDataset for vaswani, with ['corpus', 'topics', 'qrels', 'index', 'info_url', 'corpus_iter']

**we have to only take title and body only**

In [11]:
data_set = ['title', 'body']

**Defining indexer**

In [12]:
indexerfields = pt.TRECCollectionIndexer('./index', blocks = True, overwrite=True)

**set propertty**

In [13]:
pt.ApplicationSetup.setProperty("indexer.meta.forward.keys", ",".join(data_set))
pt.ApplicationSetup.setProperty("indexer.meta.reverse.keys", ",".join(data_set))
pt.ApplicationSetup.setProperty("FieldTags.process", ",".join(data_set))
pt.ApplicationSetup.setProperty("TrecDocTags.doctag", "DOC")
pt.ApplicationSetup.setProperty("TrecDocTags.idtag", "DOCNO")
pt.ApplicationSetup.setProperty("TrecDocTags.skip", "DOCHDR")


In [14]:
indexref = indexerfields.index(data.get_corpus())
indexess = pt.IndexFactory.of(indexref)

doc-text.trec:   0%|          | 0.00/0.99M [00:00<?, ?iB/s]

**Define train-query into topics and train_relevance into qrels**

In [22]:
train_query = data.get_topics()

In [21]:
train_query

,qid,query
0,1,measurement of dielectric constant of liquids ...
1,2,mathematical analysis and design details of wa...
2,3,use of digital computers in the design of band...
3,4,systems of data coding for information transfer
4,5,use of programs in engineering testing of comp...
...,...,...
88,89,tunnel diode construction and its electrical c...
89,90,electronic density of states at the surface of...
90,91,resistivity of metallic thin films related to ...
91,92,the phenomenon of radiation caused by charged ...


In [23]:
train_relevance = data.get_qrels()

In [24]:
train_relevance

,qid,docno,label
0,1,1239,1
1,1,1502,1
2,1,4462,1
3,1,4569,1
4,1,5472,1
...,...,...,...
2078,93,9875,1
2079,93,9956,1
2080,93,10497,1
2081,93,11191,1


**Perform retrieval using BM25**

In [26]:
import ir_measures
from ir_measures import *
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb

In [27]:
BM25 = pt.BatchRetrieve(indexess, wmodel="BM25")

pt.Experiment(
    [BM25], train_query, train_relevance,
    eval_metrics=[MRR@10, MRR@100, MAP@10, MAP@100, nDCG@10, nDCG@100],
    names=["BM25"]
)

,name,RR@10,RR@100,AP@10,AP@100,nDCG@10,nDCG@100
0,BM25,0.719875,0.725488,0.167809,0.272523,0.446609,0.502299


**DEfineing SDM model**

In [30]:
squence_depence = pt.rewrite.SequentialDependence(indexess)
squence_depence


SDM()

In [31]:
bm25 = pt.BatchRetrieve(indexess, wmodel="BM25")
bm25

BR(/content/index/data.properties,{'terrierql': 'on', 'parsecontrols': 'on', 'parseql': 'on', 'applypipeline': 'on', 'localmatching': 'on', 'filters': 'on', 'decorate': 'on', 'wmodel': 'BM25'},{'querying.processes': 'terrierql:TerrierQLParser,parsecontrols:TerrierQLToControls,parseql:TerrierQLToMatchingQueryTerms,matchopql:MatchingOpQLParser,applypipeline:ApplyTermPipeline,context_wmodel:org.terrier.python.WmodelFromContextProcess,localmatching:LocalManager$ApplyLocalMatching,qe:QueryExpansion,labels:org.terrier.learning.LabelDecorator,filters:LocalManager$PostFilterProcess,decorate:SimpleDecorateProcess', 'querying.postfilters': 'decorate:SimpleDecorate,site:SiteFilter,scope:Scope', 'querying.default.controls': 'wmodel:DPH,parsecontrols:on,parseql:on,applypipeline:on,terrierql:on,localmatching:on,filters:on,decorate:on', 'querying.allowed.controls': 'scope,qe,qemodel,start,end,site,scope,applypipeline', 'termpipelines': 'Stopwords,PorterStemmer'})

In [33]:
pipeline = squence_depence >> bm25
pipeline

ComposedPipeline(SDM(), BR(/content/index/data.properties,{'terrierql': 'on', 'parsecontrols': 'on', 'parseql': 'on', 'applypipeline': 'on', 'localmatching': 'on', 'filters': 'on', 'decorate': 'on', 'wmodel': 'BM25'},{'querying.processes': 'terrierql:TerrierQLParser,parsecontrols:TerrierQLToControls,parseql:TerrierQLToMatchingQueryTerms,matchopql:MatchingOpQLParser,applypipeline:ApplyTermPipeline,context_wmodel:org.terrier.python.WmodelFromContextProcess,localmatching:LocalManager$ApplyLocalMatching,qe:QueryExpansion,labels:org.terrier.learning.LabelDecorator,filters:LocalManager$PostFilterProcess,decorate:SimpleDecorateProcess', 'querying.postfilters': 'decorate:SimpleDecorate,site:SiteFilter,scope:Scope', 'querying.default.controls': 'wmodel:DPH,parsecontrols:on,parseql:on,applypipeline:on,terrierql:on,localmatching:on,filters:on,decorate:on', 'querying.allowed.controls': 'scope,qe,qemodel,start,end,site,scope,applypipeline', 'termpipelines': 'Stopwords,PorterStemmer'}))

In [34]:
pt.Experiment(
    [pipeline], train_query, train_relevance,
    eval_metrics=[MRR@10, MRR@100, MAP@10, MAP@100, nDCG@10, nDCG@100],
    names=["BM25+sd"]
)

,name,RR@10,RR@100,AP@10,AP@100,nDCG@10,nDCG@100
0,BM25+sd,0.729792,0.734333,0.170217,0.274253,0.453038,0.504336


**Query expansion methods:**

• Bo1 Divergence

• KLQueryExpansion

• RM3

• AxiomaticQE

In [38]:
Bo1_divegence = pt.rewrite.Bo1QueryExpansion(indexess)
KL_query = pt.rewrite.KLQueryExpansion(indexess)
RM3 = pt.rewrite.RM3(indexess)
AQE = pt.rewrite.AxiomaticQE(indexess)

In [39]:

pt.Experiment(
    [
            BM25,
            BM25 >> Bo1_divegence >> BM25,
            BM25 >> KL_query >> BM25,
            BM25 >> RM3 >> BM25,
            BM25 >> AQE >> BM25,
    ],
    train_query, train_relevance,
    eval_metrics=[MRR@10, MRR@100, MAP@10, MAP@100, nDCG@10, nDCG@100],
    names=["BM25", "+Bo1", "+KL", "+RM3", "+AQ"]
    )

,name,RR@10,RR@100,AP@10,AP@100,nDCG@10,nDCG@100
0,BM25,0.719875,0.725488,0.167809,0.272523,0.446609,0.502299
1,+Bo1,0.681315,0.685300,0.170858,0.280307,0.452210,0.509802
2,+KL,0.674863,0.679529,0.168336,0.278259,0.447350,0.507592
3,+RM3,0.636973,0.640660,0.160819,0.268703,0.436146,0.490788
4,+AQ,0.719875,0.725488,0.167809,0.272523,0.446609,0.502299


**rerank the documents based on the expanded**

In [40]:
pipeline = pt.FeaturesBatchRetrieve(indexess, wmodel="BM25", features=["WMODEL:Tf", "WMODEL:PL2"])

In [42]:
random_forest= RandomForestRegressor(n_estimators=5)
random_forest


RandomForestRegressor(n_estimators=5)

In [43]:
rf_pipe = pipeline >> pt.ltr.apply_learned_model(random_forest)
rf_pipe.fit(train_query.iloc[:10], train_relevance.iloc[:10])

**Repeat the same exercise :**

• Bo1 Divergence

• KLQueryExpansion

• RM3

• AxiomaticQE**

In [44]:
Bo1_divegence = pt.rewrite.Bo1QueryExpansion(indexess)
KL_query = pt.rewrite.KLQueryExpansion(indexess)
RM3 = pt.rewrite.RM3(indexess)
AQE = pt.rewrite.AxiomaticQE(indexess)
pt.Experiment(
    [
            BM25,
            BM25 >> Bo1_divegence >> rf_pipe,
            BM25 >> KL_query >> BM25 >> rf_pipe,
            BM25 >> RM3 >> BM25 >> rf_pipe,
            BM25 >> AQE >> BM25 >> rf_pipe,
    ],
    train_query, train_relevance,
    eval_metrics=[MRR@10, MRR@100, MAP@10, MAP@100, nDCG@10, nDCG@100],
    names=["BM25", "+Bo1", "+KL", "+RM3", "+AQ"]
    )

,name,RR@10,RR@100,AP@10,AP@100,nDCG@10,nDCG@100
0,BM25,0.719875,0.725488,0.167809,0.272523,0.446609,0.502299
1,+Bo1,0.074535,0.094107,0.003146,0.007769,0.027143,0.055889
2,+KL,0.042874,0.064494,0.000866,0.005792,0.012799,0.049898
3,+RM3,0.034392,0.056500,0.001537,0.006303,0.016128,0.050741
4,+AQ,0.020230,0.042893,0.001135,0.005475,0.013546,0.048344


**Using XG boost ranking**

In [45]:
xgboost_rank = xgb.sklearn.XGBRanker(objective='rank:ndcg',
      learning_rate=0.1,
      gamma=1.0,
      min_child_weight=0.1,
      max_depth=6,
      verbose=2,
      random_state=42)

xgboost_rank


XGBRanker(base_score=None, booster=None, callbacks=None, colsample_bylevel=None,
          colsample_bynode=None, colsample_bytree=None,
          early_stopping_rounds=None, enable_categorical=False,
          eval_metric=None, feature_types=None, gamma=1.0, gpu_id=None,
          grow_policy=None, importance_type=None, interaction_constraints=None,
          learning_rate=0.1, max_bin=None, max_cat_threshold=None,
          max_cat_to_onehot=None, max_delta_step=None, max_depth=6,
          max_leaves=None, min_child_weight=0.1, missing=nan,
          monotone_constraints=None, n_estimators=100, n_jobs=None,
          num_parallel_tree=None, objective='rank:ndcg', predictor=None, ...)

In [ ]:
xgboost_rank_pipe = pipeline >> pt.ltr.apply_learned_model(xgboost_rank, form="ltr")
xgboost_rank_pipe.fit(train_query, train_relevance,train_query,train_relevance)

In [47]:
sdm = pt.rewrite.SequentialDependence(indexess)
BM25 = pt.BatchRetrieve(indexess, wmodel="BM25")
pipeline = sdm >> BM25 >> xgboost_rank_pipe

In [48]:
pt.Experiment(
    [pipeline], train_query, train_relevance,
    eval_metrics=[MRR@10, MRR@100, MAP@10, MAP@100, nDCG@10, nDCG@100],
    names=["BM25+SD+LTR"]
)

,name,RR@10,RR@100,AP@10,AP@100,nDCG@10,nDCG@100
0,BM25+SD+LTR,0.034349,0.037745,0.001272,0.001649,0.008911,0.009864


In [49]:
Bo1_divegence = pt.rewrite.Bo1QueryExpansion(indexess)
KL_query = pt.rewrite.KLQueryExpansion(indexess)
RM3 = pt.rewrite.RM3(indexess)
AQE = pt.rewrite.AxiomaticQE(indexess)
pt.Experiment(
    [
            BM25,
            BM25 >> Bo1_divegence >> xgboost_rank_pipe,
            BM25 >> KL_query >> xgboost_rank_pipe,
            BM25 >> RM3 >> xgboost_rank_pipe,
            BM25 >> AQE >> xgboost_rank_pipe,
    ],
    train_query, train_relevance,
    eval_metrics=[MRR@10, MRR@100, MAP@10, MAP@100, nDCG@10, nDCG@100],
    names=["BM25", "+Bo1+Ltr", "+KL+Ltr", "+RM3+Ltr", "+AQ+Ltr"]
    )

,name,RR@10,RR@100,AP@10,AP@100,nDCG@10,nDCG@100
0,BM25,0.719875,0.725488,0.167809,0.272523,0.446609,0.502299
1,+Bo1+Ltr,0.235582,0.249344,0.017495,0.060210,0.112609,0.219773
2,+KL+Ltr,0.189495,0.213015,0.017042,0.059575,0.095937,0.219390
3,+RM3+Ltr,0.182873,0.205187,0.022995,0.069253,0.112953,0.235136
4,+AQ+Ltr,0.438633,0.449276,0.110115,0.157906,0.271871,0.352883
